# **Семинар 4. Группировка. Сводные таблицы**

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/aaparshina/FCI_22-23_data_analysis/main/data/titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## **Частотные таблицы**

Строятся как правило по категориальным данным

``` python
датафрейм['колонка'].value_counts()
```

In [ ]:
df['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [ ]:
df['Pclass'].value_counts(normalize = True)

3    0.551066
1    0.242424
2    0.206510
Name: Pclass, dtype: float64

## **Методы агрегирования данных**

**Функция агрегирования** - это метод, который принимает несколько отдельных значений и возвращает сводные данные.

### `.mean()`

In [ ]:
df['Fare'].mean() #среднее

32.204207968574636

### `.sum()`

In [ ]:
df['Fare'].sum() #сумма

28693.9493

### `.min()` and `.max()`

In [ ]:
df['Fare'].min()

0.0

In [ ]:
df['Fare'].max()

512.3292

### `.count()`

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
df['Cabin'].count() #количество не пропущенных значения в столбце

204

### `.nunique()`

In [ ]:
df['Cabin'].nunique() #количество уникальных значений в столбце

147

In [ ]:
Cabin
123
234
123
123
456
111


4

## **Группировка данных**

### Простейшая группировка


``` python
датафрейм.groupby("столбец группировки")["столбец интереса"].метод агрегирования()
```

Столбец группировки:
- Категориальные переменные

Столбец интереса:
- Количественные переменные
- Категориальные переменные

Найдем среднюю цену за билет в разных портах:

In [ ]:
df.groupby("Embarked")["Fare"].mean()

Embarked
C    59.954144
Q    13.276030
S    27.079812
Name: Fare, dtype: float64

### Несколько столбцов интереса

``` python
датафрейм.groupby("столбец группировки")[["ст. интереса 1", "ст. интереса 2", ...]].метод агрегирования()
```

Найдем среднюю цену за билет и среднй возраст людей в разных портах:

In [ ]:
df.groupby('Embarked')[['Fare', 'Age']].mean()

,Fare,Age
Embarked,,
C,59.954144,30.814769
Q,13.276030,28.089286
S,27.079812,29.445397


### Несколько столбцов группировки

``` python
датафрейм.groupby(["ст. группировки 1", "ст. группировки 2", ...])["столбец интереса"].метод агрегирования()
```

Найдем среднюю цену за билет для мужчин и женщин из разных портов:

In [ ]:
df.groupby(['Sex', 'Embarked'])['Fare'].mean()

Sex     Embarked
female  C           75.169805
        Q           12.634958
        S           38.740929
male    C           48.262109
        Q           13.838922
        S           21.711996
Name: Fare, dtype: float64

### Несколько методов агрегирования

In [ ]:
df.groupby("Embarked")["Fare"].agg(['mean', 'sum'])

,mean,sum
Embarked,,
C,59.954144,10072.2962
Q,13.276030,1022.2543
S,27.079812,17439.3988


## **C2.4**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Датасет содержит следующие признаки:

**category** - в какой категории находится блюдо (завтрак, десерты, рыба и курица и т.д)

**item** - название блюда

**calories** - количество калорий в блюде

**total_fat** - количество жиров (в процентах от дневной нормы)

**calcium** - количество кальция

**sugars** - количество углеводов

**protein** - количество белка


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/!hse_study/culture_python_andan_22_23/public_culture_22_23/sem/s11_groupby/mcdonalds.csv')
data.head()

,category,item,calories,total_fat,calcium,sugars,protein
0,Breakfast,Egg McMuffin,300,20,25,3,17
1,Breakfast,Egg White Delight,250,12,25,3,18
2,Breakfast,Sausage McMuffin,370,35,25,2,14
3,Breakfast,Sausage McMuffin with Egg,450,43,30,2,21
4,Breakfast,Sausage McMuffin with Egg Whites,400,35,25,2,21


Для какой категории **разница между минимумом и максимумом калорий** (размах) равна **760**? Напишите название, как в данных (просто скопируйте его, чтобы не опечататься):

In [ ]:
gb = data.groupby("category")["calories"].agg(["min", "max"])
gb["diff"] = gb["max"] - gb["min"]
gb[gb["diff"] == 760]

,min,max,diff
category,,,
Coffee & Tea,0,760,760


## **C6**

Создайте сводную таблицу: в рядах должны быть категории (**category**), в колонках белок (**protein**), в ячейках — среднее арифметическое значение белка по категориям.

Блюда какой категории в среднем содержат больше всего белка? В ответ запишите название категории как в данных.

In [ ]:
data.groupby("category")["protein"].mean().sort_values(ascending = False)

'Chicken & Fish'